In [ ]:
import os
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Clear CUDA cache if GPU is available
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"Initial GPU memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Initial GPU memory cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
# Training hyperparameters
arch = 'resnet101'  # Options: 'resnet18', 'resnet34', 'resnet50', 'resnet101'
batch_size = 128
test_batch_size = 128
epochs = 100
learning_rate = 0.01
momentum = 0.9
weight_decay = 5e-4
num_workers = 2
save_model = True
pretrained = False

In [ ]:
# Helper function to print GPU memory usage
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

# Helper function to clear memory
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
# Get ResNet model
def get_model(architecture, num_classes=100, use_pretrained=True):
    """
    Create a ResNet model with the specified architecture
    
    Args:
        architecture: ResNet variant (resnet18, resnet34, resnet50, or resnet101)
        num_classes: Number of output classes
        use_pretrained: Whether to use ImageNet pretrained weights
    
    Returns:
        PyTorch ResNet model
    """
    if architecture == 'resnet18':
        model = models.resnet18(weights='IMAGENET1K_V1' if use_pretrained else None)
    elif architecture == 'resnet34':
        model = models.resnet34(weights='IMAGENET1K_V1' if use_pretrained else None)
    elif architecture == 'resnet50':
        model = models.resnet50(weights='IMAGENET1K_V1' if use_pretrained else None)
    elif architecture == 'resnet101':
        model = models.resnet101(weights='IMAGENET1K_V1' if use_pretrained else None)
    else:
        raise ValueError(f"Unsupported architecture: {architecture}")
    
    # Modify the final fully connected layer to match CIFAR-100 classes
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, scheduler=None, scaler=None):
    """
    Training function for one epoch
    
    Args:
        model: PyTorch model
        device: Device to train on (cuda/cpu)
        train_loader: DataLoader for training data
        optimizer: PyTorch optimizer
        criterion: Loss function
        epoch: Current epoch number
        scheduler: Learning rate scheduler (optional)
        scaler: Gradient scaler for mixed precision training (optional)
    
    Returns:
        average_loss, accuracy
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}')
    for batch_idx, (data, target) in enumerate(progress_bar):
        # Manually clear cache to reduce memory fragmentation
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        data, target = data.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad(set_to_none=True)
        
        # Mixed precision training if scaler is provided
        if scaler is not None:
            with torch.cuda.amp.autocast():
                output = model(data)
                loss = criterion(output, target)
            
            # Use scaler for backpropagation and optimization
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        
        # Update statistics
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
        # Clean up tensors
        del output, loss, data, target
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': running_loss / (batch_idx + 1),
            'acc': 100. * correct / total
        })
    
    # Update learning rate
    if scheduler is not None:
        scheduler.step()
    
    return running_loss / len(train_loader), 100. * correct / total

def test(model, device, test_loader, criterion, scaler=None):
    """
    Evaluation function
    
    Args:
        model: PyTorch model
        device: Device to test on (cuda/cpu)
        test_loader: DataLoader for test data
        criterion: Loss function
        scaler: Gradient scaler for mixed precision (optional)
    
    Returns:
        average_loss, accuracy
    """
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc='Test')
        for data, target in progress_bar:
            # Clear cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
            data, target = data.to(device), target.to(device)
            
            # Mixed precision if scaler is provided
            if scaler is not None:
                with torch.cuda.amp.autocast():
                    output = model(data)
                    loss = criterion(output, target)
            else:
                output = model(data)
                loss = criterion(output, target)
            
            # Update statistics
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            # Clean up tensors
            del output, loss, data, target
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': test_loss / (progress_bar.n + 1),
                'acc': 100. * correct / total
            })
    
    return test_loss / len(test_loader), 100. * correct / total

# Data loading parameters
kwargs = {'num_workers': num_workers, 'pin_memory': True} if torch.cuda.is_available() else {}

# Data augmentation and preprocessing
# Option 1: Upscale to 224x224 (for transfer learning and pretrained models)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),  # Upscale to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

transform_test = transforms.Compose([
    transforms.Resize(224),  # Upscale to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

test_dataset = datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

test_loader = DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

# Create model
print(f"Using {arch} architecture")
model = get_model(arch, num_classes=100, use_pretrained=pretrained)
model = model.to(device)

# Print model structure
print(model)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum, weight_decay=weight_decay)

# Learning rate scheduler - using cosine annealing
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Initialize mixed precision training scaler
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

# For recording training history
train_losses = []
test_losses = []
train_accs = []
test_accs = []

# Save best model
best_acc = 0.0

# Training loop
for epoch in range(epochs):
    # Clear memory
    clear_memory()
    print(f"Memory state before Epoch {epoch+1}:")
    print_gpu_memory()
    
    # Train and test
    train_loss, train_acc = train(
        model, device, train_loader, optimizer, criterion, epoch, scheduler, scaler)
    
    # Clear memory between train and test
    clear_memory()
    
    test_loss, test_acc = test(model, device, test_loader, criterion, scaler)
    
    # Record history
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    # Print results
    print(f'Epoch: {epoch+1}/{epochs}')
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')
    print('-' * 70)
    
    # Save best model
    if test_acc > best_acc:
        best_acc = test_acc
        if save_model:
            # Save model
            save_path = f'cifar100_{arch}_best.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_acc': best_acc,
            }, save_path)
            print(f'Saved best model to {save_path} [Accuracy: {best_acc:.2f}%]')
    
    # Save checkpoint every 10 epochs
    if save_model and (epoch + 1) % 10 == 0:
        save_path = f'cifar100_{arch}_checkpoint_epoch{epoch+1}.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_acc': best_acc,
        }, save_path)
        print(f'Saved checkpoint to {save_path}')
    
    # Clear memory after each epoch
    clear_memory()
    print(f"Memory state after Epoch {epoch+1}:")
    print_gpu_memory()

# Plot training history
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curves')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(test_accs, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy Curves')

plt.tight_layout()
plt.savefig(f'cifar100_{arch}_training_curves.png')
plt.show()

# Load best model and evaluate
print("Evaluating best model...")
checkpoint = torch.load(f'cifar100_{arch}_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

test_loss, test_acc = test(model, device, test_loader, criterion)
print(f"Best model accuracy: {test_acc:.2f}%")

# Function to change architecture and retrain
def change_architecture(new_arch):
    """
    Function to switch to a different ResNet architecture
    
    Args:
        new_arch: New architecture to use ('resnet18', 'resnet34', 'resnet50', 'resnet101')
    
    Returns:
        New model
    """
    global arch
    arch = new_arch
    print(f"Switching to {new_arch} architecture")
    
    # Clear memory
    clear_memory()
    
    # Create new model
    new_model = get_model(new_arch, num_classes=100, use_pretrained=pretrained)
    new_model = new_model.to(device)
    
    return new_model

# Example of how to switch architecture:
# To use a different architecture, uncomment and run:
# model = change_architecture('resnet34')
# Then re-run the training loop

Using device: cuda
Initial GPU memory allocated: 479.21 MB
Initial GPU memory cached: 916.00 MB
Files already downloaded and verified
Files already downloaded and verified
Using resnet101 architecture
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): Batch

Test: 100%|██████████| 79/79 [00:05<00:00, 13.89it/s, loss=3.88, acc=10.7]


Epoch: 1/100
Train Loss: 4.3136 | Train Acc: 5.52%
Test Loss: 3.8296 | Test Acc: 10.66%
----------------------------------------------------------------------
Saved best model to cifar100_resnet101_best.pth [Accuracy: 10.66%]
Memory state after Epoch 1:
GPU memory allocated: 706.38 MB
GPU memory cached: 1182.00 MB
Memory state before Epoch 2:
GPU memory allocated: 706.38 MB
GPU memory cached: 1182.00 MB


Test: 100%|██████████| 79/79 [00:05<00:00, 13.79it/s, loss=3.55, acc=15.8]


Epoch: 2/100
Train Loss: 3.7029 | Train Acc: 12.60%
Test Loss: 3.5058 | Test Acc: 15.85%
----------------------------------------------------------------------
Saved best model to cifar100_resnet101_best.pth [Accuracy: 15.85%]
Memory state after Epoch 2:
GPU memory allocated: 705.25 MB
GPU memory cached: 1148.00 MB
Memory state before Epoch 3:
GPU memory allocated: 705.25 MB
GPU memory cached: 1148.00 MB


...

Test: 100%|██████████| 79/79 [00:05<00:00, 14.82it/s, loss=1.18, acc=72.9]


Epoch: 69/100
Train Loss: 0.0076 | Train Acc: 99.96%
Test Loss: 1.1663 | Test Acc: 72.94%
----------------------------------------------------------------------
Memory state after Epoch 69:
GPU memory allocated: 705.53 MB
GPU memory cached: 1462.00 MB
Memory state before Epoch 70:
GPU memory allocated: 705.53 MB
GPU memory cached: 1462.00 MB


Test: 100%|██████████| 79/79 [00:05<00:00, 13.90it/s, loss=1.18, acc=73.1]


Epoch: 70/100
Train Loss: 0.0077 | Train Acc: 99.96%
Test Loss: 1.1609 | Test Acc: 73.07%
----------------------------------------------------------------------
Saved checkpoint to cifar100_resnet101_checkpoint_epoch70.pth
Memory state after Epoch 70:
GPU memory allocated: 705.25 MB
GPU memory cached: 1184.00 MB
Memory state before Epoch 71:
GPU memory allocated: 705.25 MB
GPU memory cached: 1184.00 MB


Epoch 71:  91%|█████████ | 356/391 [01:39<00:09,  3.59it/s, loss=0.00726, acc=100]